In [6]:
%pip install -q amplpy

In [7]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],
    license_uuid="default",
)

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


In [14]:
%%writefile wsyz_projekt_zad1.mod

set PRODUCENCI;
set MAGAZYNY;
set WARZYWA;

param CENA_ZA_KM_DOSTAWY >= 0;

param ProdukcjaMax{PRODUCENCI, WARZYWA} >= 0;
param PojemnoscMagazynu{MAGAZYNY} >= 0;
param PopytWMagazynie{MAGAZYNY, WARZYWA} >= 0;
param Odleglosc{PRODUCENCI, MAGAZYNY} >= 0;

param KosztTransportuZaOdleglosc{p in PRODUCENCI, m in MAGAZYNY} := Odleglosc[p,m] * CENA_ZA_KM_DOSTAWY;

var x{PRODUCENCI, MAGAZYNY, WARZYWA} >= 0;

minimize KosztCalkowity:
    sum{p in PRODUCENCI, m in MAGAZYNY, w in WARZYWA}
        x[p,m,w] * KosztTransportuZaOdleglosc[p,m];

subject to LimitProdukcji {p in PRODUCENCI, w in WARZYWA}:
    sum{m in MAGAZYNY} x[p,m,w] <= ProdukcjaMax[p,w];

subject to LimitPojemnosciMagazynu {m in MAGAZYNY}:
    sum{p in PRODUCENCI, w in WARZYWA} x[p,m,w] <= PojemnoscMagazynu[m];

subject to PopytLokalny {m in MAGAZYNY, w in WARZYWA}:
   sum{p in PRODUCENCI} x[p,m,w] >= PopytWMagazynie[m,w];

Overwriting wsyz_projekt_zad1.mod


In [15]:
%%writefile wsyz_projekt_zad1.dat

set PRODUCENCI := P1 P2 P3 P4 P5 P6;
set MAGAZYNY := M1 M2 M3;
set WARZYWA := Ziemniaki Kapusta Buraki Marchew;
param CENA_ZA_KM_DOSTAWY := 5;

param ProdukcjaMax:
      Ziemniaki Kapusta Buraki Marchew :=
  P1  160       180     120    160
  P2  90        100     140    80
  P3  160       70      190    100
  P4  260       50      60     100
  P5  200       230     190    110
  P6  250       130     80     100;

param PojemnoscMagazynu :=
  M1 800
  M2 1200
  M3 750;

# Proponowany popyt roczny
param PopytWMagazynie:
      Ziemniaki Kapusta Buraki Marchew :=
  M1  233       145     116    131
  M2  349       218     175    196
  M3  218       137     109    123;

param Odleglosc:
      M1  M2  M3 :=
  P1  14  33  48
  P2  16  23  49
  P3  36  16  55
  P4  42  30  27
  P5  44  53  10
  P6  38  50  22;

Overwriting wsyz_projekt_zad1.dat


In [16]:
%%ampl_eval
reset;
model wsyz_projekt_zad1.mod;
data wsyz_projekt_zad1.dat;
option solver highs;
solve;
display KosztCalkowity;

HiGHS 1.11.0: HiGHS 1.11.0: optimal solution; objective 174145
21 simplex iterations
0 barrier iterations
KosztCalkowity = 174145

